In [103]:
import pandas as pd 
import numpy as np 
from transformers import AutoTokenizer
from transformers import pipeline

In [104]:
df = pd.read_csv('data/news_summary.csv',encoding='utf-8',encoding_errors = 'replace')

In [105]:
df.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [106]:
df = df.drop(columns=['author','date','read_more'])
df

,headlines,text,ctext
0,Daman & Diu revokes mandatory Rakshabandhan in...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika slams user who trolled her for 'divorc...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,'Virgin' now corrected to 'Unmarried' in IGIMS...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Aaj aapne pakad liya: LeT man Dujana before be...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotel staff to get training to spot signs of s...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...
...,...,...,...
4509,Rasna seeking ?250 cr revenue from snack categ...,Fruit juice concentrate maker Rasna is eyeing ...,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m..."
4510,Sachin attends Rajya Sabha after questions on ...,Former Indian cricketer Sachin Tendulkar atten...,Former cricketer Sachin Tendulkar was spotted ...
4511,Shouldn't rob their childhood: Aamir on kids r...,"Aamir Khan, while talking about reality shows ...","Aamir Khan, whose last film Dangal told the st..."
4512,"Asha Bhosle gets ?53,000 power bill for unused...",The Maharashtra government has initiated an in...,Maharahstra Power Minister Chandrashekhar Bawa...


In [107]:
##Checking null values in the dataset
df.isnull().sum()

headlines      0
text           0
ctext        118
dtype: int64

In [108]:
##Dropping the columns with null values
df = df.dropna()
df.head()

,headlines,text,ctext
0,Daman & Diu revokes mandatory Rakshabandhan in...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika slams user who trolled her for 'divorc...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,'Virgin' now corrected to 'Unmarried' in IGIMS...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Aaj aapne pakad liya: LeT man Dujana before be...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotel staff to get training to spot signs of s...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [109]:
#adding the headlines and ctext column and making a new news column
df['news'] = (df['headlines'] + '.' + df['ctext']).apply(str.lower)
df = df.drop(columns=['ctext','headlines'])
df= df.rename(columns={'text':'summary'})
df['summary']=df['summary'].apply(str.lower)
df.head()

/var/folders/5k/wmytnlns4sd2bm57pyry3xxr0000gn/T/ipykernel_22086/2204458343.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['news'] = (df['headlines'] + '.' + df['ctext']).apply(str.lower)


,summary,news
0,the administration of union territory daman an...,daman & diu revokes mandatory rakshabandhan in...
1,malaika arora slammed an instagram user who tr...,malaika slams user who trolled her for 'divorc...
2,the indira gandhi institute of medical science...,'virgin' now corrected to 'unmarried' in igims...
3,lashkar-e-taiba's kashmir commander abu dujana...,aaj aapne pakad liya: let man dujana before be...
4,hotels in maharashtra will train their staff t...,hotel staff to get training to spot signs of s...


making the dataset

splitting the data into train and test

In [116]:
from sklearn.model_selection import train_test_split
import datasets
# Converting the pandas dataset to huggingface dataset
# first split the train and test set
train_df, test_df = train_test_split(df, test_size=0.01, shuffle=True)
print("train and val shape:", train_df.shape, "test shape:",test_df.shape)
# save for every model inference
global_train_df = train_df
global_test_df = test_df
train_df = datasets.Dataset.from_pandas(train_df)
train_df = train_df.remove_columns(["__index_level_0__"])
# split train into train and val
train_df = train_df.train_test_split(test_size=0.2, shuffle=True) # split train and test
train_df["train"][0]

train and val shape: (4352, 2) test shape: (44, 2)


{'summary': "a 22-year-old man, rahat khan, has been arrested by the police in greater noida for allegedly posting a morphed image of newly elected uttar pradesh chief minister yogi adityanath on his facebook wall. a complaint was lodged against khan, who captioned the image, 'yogi hai, ya bhogi hai', by members of hindu yuva vahini which was founded by adityanath.",
 'news': '22-year-old rahat khan arrested for fb post on up cm yogi.a 22-year-old youth was arrested by the greater noida police on thursday for allegedly posting objectionable content against newly elected uttar pradesh chief minister yogi adityanath on his facebook wall.a complaint was lodged against accused rahat khan by members of the hindu yuva vahini, a youth group founded by adityanath, in dankaur police station on thursday evening.acting on the complaint, the police nabbed khan, who runs a jan suvidha kendra (public utility center) in dankaur.?rahat khan has posted morphed image of the newly elected chief minister 

In [111]:
checkpoint = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json: 100%|██████████| 2.32k/2.32k [00:00<00:00, 3.14MB/s]
spiece.model: 100%|██████████| 792k/792k [00:00<00:00, 918kB/s]
tokenizer.json: 100%|██████████| 1.39M/1.39M [00:01<00:00, 1.29MB/s]


In [112]:
prefix = "summarize: "

def preprocess(news):
    inputs = [prefix + doc for doc in news['news']]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)  # for news data
    labels = tokenizer(news['summary'], max_length=128, truncation=True)  # for summary
    model_inputs['labels'] = labels['input_ids']
    return model_inputs